In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pypdf


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install faiss-cpu

In [6]:
from langchain_community.llms import Ollama
llm = Ollama(model="codellama:7B")

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
#from langchain_community.document_loaders import WebBaseLoader
#loader = WebBaseLoader("https://fallout.fandom.com/wiki/Fallout:_New_Vegas")
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("LLM_Trial_log .pdf")
pages = loader.load_and_split()

docs = pages
#docs = loader.load()

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""
You are a verilog coder. Only output Verilog code. 
                                          
Develop a verilog file that represents the netlist for a lab on chip design

Only use one module. Use single bit wires(no [7:0] bus designations). Define internal connections at the beginning as wires.

Develop a verilog file that represents the netlist for a lab on chip design. Use component calls like the following (Zs are the only part that change): diffmix_25px_0 mixZ (.a_fluid(solnZ), .b_fluid(solnZ), .out_fluid(connectZ));
It is essential that the netlist be entirely structural, without any behavioral statements.
                                          
The goal is to take a high level design request and put it into structural verilog.

                                          
The following context may help: 

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [14]:


response = retrieval_chain.invoke({"input": "Could you make me a chip that mixes four chemicals"})
print(prompt)

print(response["answer"])


input_variables=['context', 'input'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nYou are a verilog coder. Only output Verilog code. \n                                          \nDevelop a verilog file that represents the netlist for a lab on chip design\n\nOnly use one module. Use single bit wires(no [7:0] bus designations). Define internal connections at the beginning as wires.\n\nDevelop a verilog file that represents the netlist for a lab on chip design. Use component calls like the following (Zs are the only part that change): diffmix_25px_0 mixZ (.a_fluid(solnZ), .b_fluid(solnZ), .out_fluid(connectZ));\nIt is essential that the netlist be entirely structural, without any behavioral statements.\n                                          \nThe goal is to take a high level design request and put it into structural verilog.\n\n                                          \nThe following context may help: \n\n<context>\n{con